## GAT

In [1]:
import sys
 
sys.path.insert(0, "../")

In [2]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import cudf
import torch

data = pd.read_csv(r"../extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [3]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))

In [4]:
max_Id = data["Id"].unique().max()

In [5]:
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)

In [6]:
data = pd.concat((data, gw_data))
data = cudf.from_pandas(data)

In [7]:
data = data.sort_values("Id")

In [8]:
unique_ids = data["Id"].unique()

In [9]:
data = data.reset_index(drop=True)

In [10]:
data.pop("index")

0          665587
1          665588
2          665589
3          665590
4          665591
            ...  
1819430    665582
1819431    665580
1819432    665581
1819433    665585
1819434    665586
Name: index, Length: 1819435, dtype: int64

In [25]:
from dataAnalysis.Constants import FEATURES_IN_TABLE
data[data["Label"] == "Sepsis"][FEATURES_IN_TABLE].describe().loc[["mean", "50%","std"]]

/home/dwalke/miniconda3/lib/python3.10/site-packages/cudf/core/dataframe.py:4953: FutureWarning: `datetime_is_numeric` is deprecated. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(
/home/dwalke/miniconda3/lib/python3.10/site-packages/cudf/core/series.py:3120: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


,Age,HGB,WBC,RBC,MCV,PLT
mean,66.082386,6.953003,15.375617,3.781871,89.894562,212.805601
50%,68.000000,7.000000,12.800000,3.800000,89.300000,196.000000
std,14.289994,1.851757,22.045940,1.035372,8.495609,142.186344


In [26]:
data[data["Label"] == "Control"][FEATURES_IN_TABLE].describe().loc[["mean", "50%","std"]]

/home/dwalke/miniconda3/lib/python3.10/site-packages/cudf/core/dataframe.py:4953: FutureWarning: `datetime_is_numeric` is deprecated. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(
/home/dwalke/miniconda3/lib/python3.10/site-packages/cudf/core/series.py:3120: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


,Age,HGB,WBC,RBC,MCV,PLT
mean,58.719931,7.491711,8.397616,4.064619,88.675829,241.053003
50%,61.000000,7.700000,7.510000,4.170000,88.500000,233.000000
std,18.318832,1.504487,7.314774,0.819608,6.287799,112.970617


## Creating edge index

In [30]:
# import numpy as np
# import cupy as cp
# import time

# source_edge_index = cp.array([], dtype= cp.int32)
# target_edge_index = cp.array([], dtype= cp.int32)

# start = time.time()
# i = 0
# for Id, group in data.groupby("Id"):
#     indices = group.index
#     indices = cp.expand_dims(indices.values, axis = 1)
#     target = cp.repeat(indices, indices.shape[0], axis = 1)
#     source = target.transpose().flatten()
#     target = target.flatten()

#     source_edge_index = cp.concatenate((source_edge_index, source))
#     target_edge_index = cp.concatenate((target_edge_index, target))
#     i+=1
#     if i % 1000 == 0:
#         print(f"{str(i / unique_ids.shape[0] * 100)} %")

# print(time.time() - start )
# print(source_edge_index.shape)
# print(target_edge_index.shape)

# edge_index = np.asarray([np.asarray(source_edge_index.get()), np.asarray(target_edge_index.get())])
# edge_index = torch.tensor(edge_index)
#pd.DataFrame(edge_index.numpy().transpose()).to_csv("edge_index_sorted_ids_w_gw.csv", index=False)

   Age Sex  HGB   WBC   RBC   MCV    PLT
4   63   W  7.4  20.9  3.96  86.6  189.0
5   63   W  6.5   7.1  3.56  90.2  349.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
    Age Sex  HGB   WBC   RBC    MCV    PLT
7    64   W  5.6  10.4  2.82   99.6  230.0
8    64   W  4.2   4.3  2.11  103.3  329.0
9    64   W  5.1   5.6  2.62  100.0  346.0
10   64   W  5.5   6.4  2.77  102.5  413.0
11   64   W  5.3   7.2  2.71  103.0  194.0
12   64   W  5.2   5.5  2.56  102.3  208.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
    Age Sex  HGB  WBC   RBC   MCV    PLT
15   26   W  7.5  5.4  4.37  82.2  274.0
16   26   W  6.5  6.7  3.85  81.3  214.0
17   26   W  7.7  6.3  4.49  82.6  282.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
    Age Sex  HGB  WBC   RBC   MCV    PLT
22   76   M  8.9  9.5  4.47  92.2  215.0
23   76   M  8.5  4.5  4.34  95.9  192.0
24   76   M  8.1  6.4  4.11  92.7  186.0
Empty DataFrame
Columns: [Age, Sex,

Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB   WBC   RBC   MCV    PLT
257   49   M  8.2  14.4  4.40  91.8  282.0
258   49   M  7.4  10.9  3.91  90.8  233.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB   WBC   RBC   MCV    PLT
259   30   W  7.2   8.2  3.92  88.3  175.0
260   30   W  6.9  12.1  3.79  86.3  153.0
261   30   W  7.3   9.8  4.01  87.0  189.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB  WBC   RBC   MCV    PLT
267   78   M  6.9  6.4  3.60  96.4  316.0
268   78   M  7.0  6.4  3.63  97.0  306.0
269   78   M  6.2  6.2  3.33  96.4  249.0
270   78   M  6.3  6.3  3.29  93.9  273.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB  WBC   RBC   MCV    PLT
272   76   W  8.8  7.9  4.32  96.8  257.0
273   76   W  8.8  8.1  4.39  96.6  257.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age

     Age Sex  HGB   WBC   RBC   MCV    PLT
443   29   W  5.9  15.5  3.32  84.0  215.0
444   29   W  6.8  17.0  3.73  85.3  230.0
445   29   W  6.2  15.0  3.45  84.3  250.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB   WBC   RBC   MCV    PLT
451   65   W  7.4  16.6  4.95  77.0  209.0
452   65   W  7.5   9.3  5.10  77.3  164.0
453   65   W  6.4   8.9  4.45  77.1  185.0
454   65   W  6.6  10.6  4.48  76.8  225.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB  WBC   RBC   MCV    PLT
456   73   W  8.3  6.9  4.21  97.6  128.0
457   73   W  7.9  5.2  3.98  97.2  125.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB  WBC   RBC   MCV    PLT
460   74   M  5.5  2.6  2.90  92.8   74.0
461   74   M  6.4  5.5  3.33  91.9  203.0
462   74   M  6.3  1.3  3.31  92.1  126.0
463   74   M  5.3  4.9  2.76  92.0   68.0
464   74   M  6.0  2.5  3.16  92.4   96.0
465   74   M  6.1  4.2

Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB  WBC   RBC   MCV    PLT
675   77   M  4.3  4.3  2.21  87.3  253.0
676   77   M  3.9  5.4  2.08  88.9  268.0
677   77   M  4.4  5.7  2.31  89.6  384.0
678   77   M  5.2  5.9  2.71  86.3  253.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB  WBC   RBC   MCV    PLT
680   32   W  8.3  6.3  4.55  86.8  265.0
681   32   W  8.7  3.9  4.67  89.1  186.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB   WBC   RBC   MCV    PLT
682   66   M  6.3  10.9  3.35  86.3  180.0
683   66   M  7.3  10.0  3.91  87.0  167.0
684   66   M  8.9   8.8  4.73  87.5  199.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Sex  HGB  WBC   RBC   MCV    PLT
685   78   W  7.3  7.2  4.17  81.5  222.0
686   78   W  6.9  6.4  3.97  83.4  219.0
Empty DataFrame
Columns: [Age, Sex, HGB, WBC, RBC, MCV, PLT]
Index: []
     Age Se

     Age Sex  HGB  WBC   RBC   MCV    PLT
908   85   M  6.8  5.5  3.80  86.6  176.0
909   85   M  6.9  7.2  3.79  87.1  201.0
910   85   M  6.1  4.8  3.40  85.0  153.0
911   85   M  6.8  7.0  3.76  85.9  183.0
912   85   M  6.2  5.1  3.40  85.9  155.0
913   85   M  7.1  7.1  4.01  87.0  207.0
914   85   M  7.2  7.0  4.04  87.9  196.0


KeyboardInterrupt: 

In [12]:
import torch

In [13]:
import torch
edge_index = torch.tensor(pd.read_csv("edge_index_sorted_ids_w_gw.csv", header=None, skiprows=1).values.transpose(), dtype=torch.long)

In [15]:
edge_index.shape

torch.Size([2, 12698987])

In [16]:
from dataAnalysis.Constants import SEX_CATEGORY_COLUMN_NAME, SEX_COLUMN_NAME, FEATURES
data[SEX_CATEGORY_COLUMN_NAME] = data.loc[:, SEX_COLUMN_NAME] =="W"

In [17]:
data[SEX_CATEGORY_COLUMN_NAME] = data[SEX_CATEGORY_COLUMN_NAME].astype("int8")
data["Label"] = data["Label"] == "Sepsis"
data["Label"] = data["Label"].astype("int8")

In [18]:
X_features = torch.tensor(data[FEATURES].values.get()).type(torch.float32)
y = torch.tensor(data["Label"].values.get()).type(torch.long)

In [19]:
from dataAnalysis.FeatureImportance import normalize

X_features = normalize(X_features)

In [20]:
X_new_features = X_features

In [21]:
X_new_features.shape

torch.Size([1819435, 7])

In [22]:
import numpy as np
def ratio_bool_switch(tensor, ratio = 0.8):
    random = np.random.uniform(0, 1 ,tensor.shape[0])
    val_ratio_mask = (random >= 0.8)
    train_ratio_mask = (random < 0.8)
    val_mask = np.logical_and(tensor.tolist(), val_ratio_mask.tolist())
    train_mask = np.logical_and(tensor.tolist(), train_ratio_mask.tolist())
    return torch.from_numpy(train_mask).type(torch.bool), torch.from_numpy(val_mask).type(torch.bool)

In [23]:
from torch_geometric.data import Data

train_mask_ser = data["Set"] != "Validation"
train_mask, val_mask = ratio_bool_switch(train_mask_ser.values)
test_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values.get(), (data["Center"] == "Leipzig").values.get())).type(torch.bool)
test_gw_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values.get(), (data["Center"] == "Greifswald").values.get())).type(torch.bool)
graph = Data(x=X_new_features, train_mask = train_mask, test_mask=test_mask, val_mask=val_mask, y= y, edge_index=edge_index,
             test_gw_mask = test_gw_mask)

In [24]:
test_gw_mask.sum() + test_mask.sum()+val_mask.sum()+train_mask.sum()

tensor(1819435)

In [25]:
graph

Data(x=[1819435, 7], edge_index=[2, 12698987], y=[1819435], train_mask=[1819435], test_mask=[1819435], val_mask=[1819435], test_gw_mask=[1819435])

In [26]:
graph = graph.cpu()

In [27]:
from torch_geometric.loader import NeighborLoader

kwargs = {
    "num_neighbors":[-1] * 2,
    "batch_size":50_000
}
loader = NeighborLoader(
    graph,
    input_nodes=graph.train_mask,
    **kwargs
)
val_loader = NeighborLoader(
    graph,
    input_nodes=graph.val_mask,
    **kwargs
)

In [28]:
device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

# graph = graph.to(device)
WEIGHT = torch.tensor([664*4/5])#
WEIGHT = WEIGHT.to(device)

print("Data shifted to the device " + str(device))

/home/dwalke/miniconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Data shifted to the device cuda:2


In [30]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv,GATv2Conv, GINConv, global_add_pool
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES
from torch.nn import Linear, ReLU, Sequential
from torch.nn import BatchNorm1d as BatchNorm

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(GraphNeuralNetwork, self).__init__()
        input_dim = len(FEATURES)      
        
        HEADS = 5
        
        conv_1= GATConv(input_dim, hidden_dim,heads=HEADS, add_self_loops = False)
        conv_end = GATConv((-1,-1), out_channels,add_self_loops = False)
        
        self.conv_1 = conv_1
        self.conv_end = conv_end
        

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = x.type(torch.float)
        x = self.conv_1(x, edge_index)
        x = F.normalize(x, p=2., dim=-1)
        x = torch.relu(x)
        x = self.conv_end(x, edge_index)
        return x
            
    def predict_proba(self, graph, mask):
        with torch.inference_mode():
            self.eval()
            logits = self.forward(graph)
            scores = torch.sigmoid(torch.squeeze(logits[mask]))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
            return torch.transpose(proba_predict, 0, 1)
            
    def predict(self, graph, mask):
        return torch.round(self.predict_proba(graph, mask)[:, 1])

In [31]:
import torch 
from tqdm.notebook import tqdm
class ModelWrapper():
    def __init__(self, graph):
        self.LEARNING_RATE = 3e-4
        self.MAX_EPOCHS = 10000 #40000

        self.model = GraphNeuralNetwork(hidden_dim = 128, out_channels=1) 
        self.model = self.model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LEARNING_RATE,betas=(0.9, 0.999), eps=1e-08)
        self.graph = graph
        
        self.last_loss = 0
        self.increased_loss = 0
        self.BREAKING_THRESHOLD = 5 
        self.val_loss = []
        self.train_loss = []
        self.epochs = self.MAX_EPOCHS
    
    def validate(self):
        with torch.inference_mode():
            self.model.eval()
            acc_loss = 0
            batch_size = 0
            for batch, graph in enumerate(val_loader):
                graph = graph.to(device)
                out = self.model(graph)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), graph.y.type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
            avg_loss = acc_loss / batch_size
            self.val_loss.append(avg_loss)
            if avg_loss > self.last_loss:
                self.increased_loss += 1
            else:
                self.increased_loss = 0
            self.last_loss = avg_loss

    def train(self):
        for epoch in tqdm(range(self.MAX_EPOCHS)):
            acc_loss = 0
            batch_size = 0
            for batch, graph in enumerate(loader):
                graph = graph.to(device)
                self.model.train()
                self.optimizer.zero_grad()
                out = self.model(graph)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), graph.y.type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
                loss.backward()
                self.optimizer.step()
            self.train_loss.append(acc_loss/batch_size)
            self.validate() 

            if self.increased_loss >= self.BREAKING_THRESHOLD:
                self.epochs = epoch + 1
                print(f"Breaked at {str(epoch)}")
                break
            
    def get_model(self):
        return self.model    
    
    def plot_loss(self):
        plt.plot(range(len(self.train_loss)), self.train_loss, 'g', label='Training loss')
        plt.plot(range(len(self.val_loss)), self.val_loss, 'y', label='Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [32]:
import time
from dataAnalysis.Metrics import Evaluation
train_times = []
eval_int_times = []
eval_ext_times = []
models= []
dfs = []
for i in range(100):
    graph = graph.to(device)
    model_wrapper = ModelWrapper(graph)
    
    start = time.time()
    model_wrapper.train()
    print(start -time.time())
    train_times.append(start -time.time())
    
    model = model_wrapper.get_model()
    graph = graph.cpu()
    model = model.cpu()
    models.append(model)
      
    start = time.time()
    y_dict = Evaluation.create_y_dict(model.predict(graph, test_mask), model.predict_proba(graph, test_mask) , graph.y[test_mask])
    eval_int_times.append(time.time() - start)
    
    start = time.time()
    y_dict_gw = Evaluation.create_y_dict(model.predict(graph, test_gw_mask), model.predict_proba(graph, test_gw_mask) , graph.y[test_gw_mask])
    eval_ext_times.append(time.time() - start)
    df = Evaluation.get_df_metrics_from_pred(y_dict, y_dict_gw)
    print(df)
    dfs.append(df)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [33]:
model = model_wrapper.get_model()

In [ ]:
model_wrapper = ModelWrapper(graph)
model_wrapper.train()

In [ ]:
model_wrapper.plot_loss()

In [34]:
model = model_wrapper.get_model()
graph = graph.cpu()
model = model.cpu()

In [35]:
from dataAnalysis.Metrics import Evaluation

y_dict = Evaluation.create_y_dict(model.predict(graph, test_mask), model.predict_proba(graph, test_mask) , graph.y[test_mask])
y_dict_gw = Evaluation.create_y_dict(model.predict(graph, test_gw_mask), model.predict_proba(graph, test_gw_mask) , graph.y[test_gw_mask])

In [ ]:
from dataAnalysis.Metrics import Evaluation

Evaluation.plot_confusion_matrix_from_pred(model.predict(graph, test_mask), graph.y[test_mask])
Evaluation.plot_confusion_matrix_from_pred(model.predict(graph, test_gw_mask), graph.y[test_gw_mask])
Evaluation.get_df_metrics_from_pred(y_dict, y_dict_gw)

In [ ]:
from IPython.display import Audio
sound_file = './finish_sound.mp3'
Audio(sound_file, autoplay=True)

In [56]:
##TODO:
# - change weight
# - gradients as features
# - different learning rate and number of layers
# - general architecture- less neurons but linear layer before and after convs? 
#       -adding hidden layers with each others for more stable training (guaranteed only when hidden dim is euqal betwene layers)
# - GATv2
# - MIMIC und Greifswald
# - more features

In [ ]:
import time

for i in range(10000):
    time.sleep(60)
    print("Sleeping")

Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
S